**Wesleyan University ASTR 210**

# Homework 2

Work through this Python notebook, writing your own code where indicated to solve the problems.  When you are finished, save this document and upload the completed notebook to the course Moodle.  

We will grade your work by executing the cells in order.  I suggest that you do the same thing before submitting just to make sure everything works the way you think it should, in case you accidentally ran things out of order while you were working.  To check your work, go to Kernel -> Restart Kernel and Run All Cells, then examine the output of each cell.

In this homework, you will analyze a catalog of the brightest stars in the sky to see how their properties like color, brightness, and spectral type relate to one another.  There are a few major steps to this analysis:
- Importing the data from a .txt file and turning it into a series of numpy arrays,
- Selecting the subset of stars which have a measured parallax,
- Using parallax to calculate distance,
- Using distance and apparent magnitude to calculate absolute magnitude,
- Separating stars by spectral type, and
- Measuring the *number* of stars of each type, their *median color*, and their *median absolute magnitudes* (i.e., intrinsic luminosities).

I have left my outputs after each line so that you know what you're shooting for.  At the end, you should find a few interesting trends!

**(2 points)** Import the numpy package and assign it the alias "np":

In [1]:
import numpy as np # YOUR CODE HERE

## Load a star catalog into numpy arrays (12 points)

In this homework, we will be working with an actual star catalog which is larger and more complex than the dictionary we made last time.  In addition to this python notebook, you should have downloaded a file called "starcatalog.txt" and put it in your data folder.  Open it in another window so that you can see what the data actually looks like.  You will see 8 columns:
- name: The name of the star in this catalog
- alt_name: Another name it might have, if it is a well-known star (blank otherwise)
- vmag: Visual (apparent) magnitude
- multiple: W if it is a system of multiple stars, blank if a single star
- class: What type of object it is.  (I selected only objects of class "STAR" when making the catalog.)
- bv_color: Color, measured as magnitude in the B band - magnitude in the V band
- parallax: Parallax in arcseconds, if it has been measured (blank otherwise)
- spect_type: Spectral type

We can load this data into memory using the loadtxt() function from the numpy package.  Below is a mostly-complete command for doing so.  It is only missing the first argument, which indicates the path to the data file from the location of this notebook file.  (The other arguments tell python how to parse the file, e.g. to skip the first line which contains the column headings, that the '|' character separates the columns, and that it should interpret the data as strings rather than numerical data types.)

**(4 points)** Replace PATH with a string for the **relative** path to the data file.  (Hint: remember our discussion about navigating directories using the command line.  This notebook file is inside the "code" folder, while the data file is inside the "data" folder.  How do you get there from here?  Your path should end with the name of the file, starcatalog.txt.)

In [2]:
catalog = np.loadtxt('../data/starcatalog.txt', skiprows=1, delimiter='|', dtype='str')

We now have a two-dimensional numpy array of strings.  The command below prints out the first 5 rows, just so you can see what it looks like.

In [3]:
catalog[:5,:]

array([['', 'HR 7228', 'Sig Oct   ', ' 5.47', '        ', 'STAR F0III',
        '0.27    ', '        ', 'F0III             ', ''],
       ['', 'HR 8294', '          ', ' 6.57', '        ', ' STAR F0IV',
        '0.28    ', '        ', 'F0IV-V            ', ''],
       ['', 'HR 5491', '          ', ' 6.48', '        ', '   STAR A0',
        '0.30    ', '        ', 'Am                ', ''],
       ['', 'HR 6721', 'Chi Oct   ', ' 5.28', '        ', 'STAR K3III',
        '1.28    ', '        ', 'K3III             ', ''],
       ['', 'HR 6133', '          ', ' 6.57', '        ', 'STAR G5III',
        '0.91    ', '        ', 'G5III             ', '']], dtype='<U18')

As you can see, real data can be a bit of a mess.  Some entries are missing, there's a lot of random whitespace that we don't need, etc.  Below I have written some code using list comprehensions to clean things up a bit and separate out the data so that we have a collection of 1D arrays instead, one for each column.  (You're welcome!)  You do not need to completely understand what is going on in the block below, just execute it for now.  (However, you do possess most of the tools to understand it, if you are interested.)

In [4]:
name = np.array([x.strip() for x in catalog[:,1]])
altname = np.array([x.strip() for x in catalog[:,2]])
vmag = np.array([float(x.strip()) for x in catalog[:,3]])
multiple = np.array([x.strip() for x in catalog[:,4]])
color = np.array([float(x.strip()) if len(x.strip()) > 0 else -1. for x in catalog[:,6]])
parallax = np.array([float(x.strip()) if len(x.strip()) > 0 else -1 for x in catalog[:,7]])
spectral_type = np.array([x[0] for x in catalog[:,8]])

We now have 7 arrays to play with:
- name: String array of names
- altname: String array of alternate names (empty string if no alternate name)
- vmag: Float array of visual magnitudes
- multiple: String array which is 'W' if a multiple system and an empty string otherwise
- color: Float array of B-V color, if measured (-1 otherwise)
- parallax: Float array of parallax, if measured (-1 otherwise)
- spectral_type: String array of spectral type, including only the first character ('O', 'A', etc.)

**(4 points)** Print out the first 15 elements of each of the 7 arrays above.  Then, take a look at their contents and see how they relate to the descriptions I gave above.  (You do not have to write anything, I just want you to take a moment to examine the outputs here.)

In [5]:
print(name[:15])
print(altname[:15])
print(vmag[:15])
print(multiple[:15])
print(color[:15])
print(parallax[:15])
print(spectral_type[:15])
# YOUR CODE HERE

['HR 7228' 'HR 8294' 'HR 5491' 'HR 6721' 'HR 6133' 'HR 8862' 'HR 2848'
 'HR 6139' 'HR 4709' 'HR 8505' 'HR 5084' 'HR 3678' 'HR 4595' 'HR 1271'
 'HR 6552']
['Sig Oct' '' '' 'Chi Oct' '' 'Tau Oct' '' '' '' 'Ups Oct' 'Kap Oct'
 'Zet Oct' '' '' '']
[5.47 6.57 6.48 5.28 6.57 5.49 6.47 6.04 6.33 5.77 5.58 5.42 6.05 6.41
 6.45]
['' '' '' '' '' '' '' '' '' '' '' '' 'W' 'W' '']
[ 0.27  0.28  0.3   1.28  0.91  1.27  0.42  0.05  1.08  1.02  0.18  0.31
  1.29 -0.01  0.44]
[-1.    -1.    -1.    -1.    -1.    -1.    -1.    -1.    -1.    -1.
 -1.    -1.    -1.     0.008 -1.   ]
['F' 'F' 'A' 'K' 'G' 'K' 'F' 'A' 'K' 'K' 'A' 'A' 'K' 'B' 'F']


**(4 points)** The code above just printed out the data for the first 15 stars, but there are way more stars than that in the catalog!  Write a line of code below which prints out the number of stars in the catalog (by e.g., measuring the length of one of the arrays).  (Hint: you can check this against the actual data file - but remember that we skipped the first line of the file when we read in the data.)

In [6]:
print (len(name))# YOUR CODE HERE

8774


## Explore a small subset of the data (28 points)

Before we tackle the whole huge dataset, let's explore with a smaller set of 10 stars.  Stars 500-509 are pretty interesting!

**(4 points)** Create three smaller arrays name_subset, altname_subset, and multiple_subset which contain elements 500-509 of name, altname, and multiple respectively.  (These should each have length 10.)

In [7]:
name_subset = name[500:510] # YOUR CODE HERE
altname_subset = altname[500:510] # YOUR CODE HERE
multiple_subset = multiple[500:510] # YOUR CODE HERE

In [10]:
len(name_subset)

10

In [8]:
print(name_subset)
print(altname_subset)
print(multiple_subset)

['HR 4537' 'HR 5939' 'HR 833' 'HR 6796' 'HR 5241' 'HR 6030' 'HR 2337'
 'HR 6745' 'HR 5623' 'HR 5666']
['' '' 'Gam Hor' '' '' 'Del TrA' '' 'Pi  Pav' '' 'Eps Cir']
['' '' 'W' 'W' '' 'W' '' '' '' '']


As you can see, some stars have alternate names and some do not, and some are multiples while some are singles.  Below we will build a loop which will write out the properties of each star in our subset.

**(4 points)** Write a *for* loop which iterates over the elements of name_subset and prints them out one by one.

In [9]:
for i in name_subset:
    print (i)# YOUR CODE HERE

HR 4537
HR 5939
HR 833
HR 6796
HR 5241
HR 6030
HR 2337
HR 6745
HR 5623
HR 5666


**(5 points)** Now use the enumerate() function to also generate an index each time the loop runs, and print out the sentence "*STARNAME* is at index *INDEX* in the array." where *STARNAME* and *INDEX* are the values of the loop variables.

In [10]:
for i, sub_name in enumerate(name_subset):
    print (f'{sub_name} is at index {i} in the array') # YOUR CODE HERE

HR 4537 is at index 0 in the array
HR 5939 is at index 1 in the array
HR 833 is at index 2 in the array
HR 6796 is at index 3 in the array
HR 5241 is at index 4 in the array
HR 6030 is at index 5 in the array
HR 2337 is at index 6 in the array
HR 6745 is at index 7 in the array
HR 5623 is at index 8 in the array
HR 5666 is at index 9 in the array


**(5 points)** We can use that same index to retrieve the corresponding element of altname_subset.  Change the print statement to instead read, "*STARNAME* has alternate name *ALTNAME*"

In [11]:
for i, sub_name in enumerate(name_subset):
    print (f'{sub_name} has alternate name {altname_subset[i]}') # YOUR CODE HERE

HR 4537 has alternate name 
HR 5939 has alternate name 
HR 833 has alternate name Gam Hor
HR 6796 has alternate name 
HR 5241 has alternate name 
HR 6030 has alternate name Del TrA
HR 2337 has alternate name 
HR 6745 has alternate name Pi  Pav
HR 5623 has alternate name 
HR 5666 has alternate name Eps Cir


As you can see, many stars do not have an alternate name, so that sentence doesn't make any sense! 

**(5 points)** Add an *if* statement inside your loop so that the sentence only prints out if the star has an alternate name (i.e., if the length of the altname is greater than zero).

In [12]:
for i, sub_name in enumerate(name_subset):
    if len(altname_subset[i]) > 0:
        print (f'{sub_name} has alternate name {altname_subset[i]}')# YOUR CODE HERE

HR 833 has alternate name Gam Hor
HR 6030 has alternate name Del TrA
HR 6745 has alternate name Pi  Pav
HR 5666 has alternate name Eps Cir


**(5 points)** Now let's do another version of the loop which says whether each system is a single or multiple.  Use an *if/else* statement inside the loop so that it prints "*STARNAME* is a system of multiple stars" if the corresponding value of multiple_subset is 'W', and "*STARNAME* is a single star" otherwise.

In [13]:
for i, sub_name in enumerate(name_subset):
    if  multiple_subset[i] == 'W':
        print (f'{sub_name} is a system of multiple stars')
    else:
        print (f'{sub_name} is a single star') # YOUR CODE HERE

HR 4537 is a single star
HR 5939 is a single star
HR 833 is a system of multiple stars
HR 6796 is a system of multiple stars
HR 5241 is a single star
HR 6030 is a system of multiple stars
HR 2337 is a single star
HR 6745 is a single star
HR 5623 is a single star
HR 5666 is a single star


## Calculate distances and absolute magnitudes for stars with parallaxes (26 points)

Our catalog contains information about the apparent visual magnitude and parallax of each star, if it has been measured.  From these pieces of information, we can calculate the distance to the star and its absolute magnitude.  Let's start with just a single star: Beta Doradus, at index 574 in our catalog.  The block of code below retrieves some of the information about Beta Doradus from the catalog and saves it to individual variables.

(One quick note: the values in the catalog do not match the values on Wikipedia - it's an old catalog - so don't use the internet to gauge whether your answer is correct!)

In [14]:
bd_name = name[574]
bd_altname = altname[574]
bd_vmag = vmag[574]
bd_parallax = parallax[574]
print(bd_name, bd_altname, str(bd_vmag), str(bd_parallax))

HR 1922 Bet Dor 3.76 0.012


A star's *parallax* is how much it appears to move in the sky over the course of a year as the Earth travels around the Sun.  The closer a star is, the more parallax we will observe.  Mathematically, distance $d$ and parallax angle $\theta$ are related as

$$ d = 1/\theta $$

where $d$ is in units of parsecs and $\theta$ is in units of arcseconds.  (As an aside, this is the origin of the word parsec: an object which is one **parsec** away has a **par**allax angle of one arc**sec**ond.)

**(3 points)** Calculate the distance to Beta Doradus in parsecs and save it to the variable bd_distance.

In [15]:
# YOUR CODE HERE
bd_distance = 1/bd_parallax
print (bd_distance)

83.33333333333333


As we explored in the previous homework, a star's apparent magnitude $m$ (brightness viewed from Earth), absolute magnitude $M$ (intrinsic brightness), and distance $d$ are related to one another: the further away the star is, the fainter it will appear.  The expression for this relationship is

$$M = m - 5 \log_{10}(d/10)$$

where $d$ is in parsecs.

**(3 points)** Calculate the absolute magnitude of Beta Doradus and save it to the variable bd_absmag.  (Hint: you can use the numpy function log10().)

In [16]:
# YOUR CODE HERE
bd_absmag = bd_vmag - 5*np.log10(bd_distance/10)
print(bd_absmag)

-0.8440937697618756


Now, let's apply these calculations to the whole catalog.  Not every star has a parallax measured, however, so first we'll have to select just the subsample of stars that do have one.  Stars that do *not* have a measured parallax will have a -1 in the parallax array, so we can pick out just the stars with parallaxes by searching for positive values.

**(4 points)** Create a Boolean array has_parallax which is True if the value of parallax is greater than zero, and False otherwise.

In [17]:
has_parallax = parallax > 0

**(4 points)** What fraction of the stars in the catalog have a measured parallax?  Calculate and print this fraction below. 

(Hint: the *fraction* of stars with a measured parallax is the *number* of stars which have a measured parallax, divided by the *total number* of all stars.)

(Note: if you find yourself tempted to use a *loop* here, **resist the urge**!  Use **array operations**!  In fact, *you should not need to use a loop again* until the very last block of code in this assignment.)

In [18]:
len(name[has_parallax])/len(parallax) # YOUR CODE HERE

0.3471620697515386

**(4 points)** Use has_parallax to select the subsets of the name, vmag, color, parallax, and spectral_type arrays corresponding to stars that have a measured parallax, and save those subsets as name_hp, vmag_hp, color_hp, parallax_hp, and spectral_type_hp respectively.

(The 'hp' suffix is meant to indicate 'has parallax.'  So 'name_hp' is 'the names of the stars that have a measured parallax,' 'vmag_hp' is 'the visual magnitudes of the stars that have a measured parallax,' and so on.)

In [19]:
name_hp = name[has_parallax]
vmag_hp = vmag[has_parallax]
color_hp = color[has_parallax]
parallax_hp = parallax[has_parallax]
spectral_type_hp = spectral_type[has_parallax]

**(4 points)** Use vmag_hp and parallax_hp to calculate the distances and absolute magnitudes of the stars with measured parallaxes, and save them to new arrays named distance_hp and absmag_hp.

In [20]:
distance_hp = 1/parallax_hp
absmag_hp = vmag_hp - 5*np.log10(distance_hp/10)

**(4 points)** Use np.max() and np.min() to print out the distances to the closest and furthest stars in the catalog that have a parallax.

In [21]:
print(np.max(distance_hp), np.min(distance_hp))

1000.0 1.3315579227696406


(Note: if you find the distance to the furthest star to be suspiciously round, it is because the parallaxes in the catalog only go out to 3 decimal places.)

## Separate stars by spectral type and calculate median properties (30 points)

The "spectral type" of a star tells us something about its physical properties - O stars are the brightest, bluest, and most massive stars, while K and M dwarf stars are faint and red.  Confusingly, they are not alphabetically ordered from brightest to dimmest, but go in the order OBAFGKM.  (The traditional mnemonic for remembering this is "Oh, Be A Fine Girl/Guy, Kiss Me," but this is slowly being abandoned in favor of less-creepy alternatives like "Only Bad Astronomers Forget Generally Known Mnemonics.")  Below, we will pick out subsets of stars of each type and calculate their median colors and absolute magnitudes to see how those properties change as a function of spectral type. 

**(3 points)** The Sun is a very mediocre star, with spectral type G.  How many stars in the (whole) catalog have spectral type G?  Calculate and print this number below.

In [22]:
is_g = spectral_type == 'G'
len(spectral_type[is_g])

1145

**(3 points)** How many stars *that have measured parallaxes* have spectral type G?  

Note: from this question onwards you should always be using the X_hp arrays (the subset with measured parallaxes) because we'll be calculating absolute magnitudes, which you can only do if you know the distance from the parallax.

In [23]:
is_g_hp = spectral_type_hp == 'G'
len(spectral_type_hp[is_g_hp])

555

**(4 points)** Define a Boolean array is_typeG_hp which is True when the stars in the X_hp arrays are of spectral type G, and False otherwise.  (You probably already did this in the previous question, without assigning it a name.)  To check your work, you should verify that your array is_typeG_hp is the same length as your other X_hp arrays.

In [24]:
is_typeG_hp = is_g_hp
print(len(is_typeG_hp))
print(len(parallax_hp))

3046
3046


"Color" in astronomy is generally defined as a comparison of an object's brightness at two different wavelengths of light.  If it is brighter at short wavelengths than at long wavelengths we say it is "bluer," and if it is brighter at longer wavelengths we say it is "redder."  In this catalog, color is defined as the difference in magnitude in the B (blue) band and V (visual) band.  Because magnitudes are silly, bluer colors correspond to *lower* values of B-V.

**(4 points)** Use the np.median() function, color_hp, and is_typeG_hp to find the median B-V color of all the stars with parallaxes in the catalog which are spectral type G.

In [25]:
bv_hp_typeG = color_hp[is_typeG_hp]
np.median(bv_hp_typeG)

np.float64(0.87)

**(4 points)** Find the median absolute magnitude of all the stars with parallaxes in the catalog which are spectral type G.

In [26]:
absmag_G_hp = vmag_hp[is_typeG_hp] - 5*np.log10(distance_hp[is_typeG_hp]/10)
print(np.median(absmag_G_hp))

2.1110964736695967


**(12 points)** Now let's combine all of these things into a loop over all the spectral types!  The first line in the block below defines a list of spectral types named alltypes.  Write a *for* loop over this list of spectral types, which does the following things each time the loop is executed:
- Sets the values in a Boolean array to indicate whether the stars with parallaxes are of that type (like is_typeG_hp, but for type O the first time the loop runs, type B the second time the loop runs, and so on),
- Prints "There are *N* stars of type *TYPE* with parallaxes" where *TYPE* is the loop variable and *N* is the total number of stars with parallaxes that are of that type
- Prints "Their median color is *COLOR*" where *COLOR* is the median B-V color of stars of that type
- Prints "Their median magnitude is *ABSMAG*" where *ABSMAG* is the median absolute magnitude of stars of that type
- Prints an empty line to put some space before the next loop

(Hint: if you find yourself using for-loops-within-for-loops here, you are making it too complicated!  The easy way to do this is to *use the Boolean array to select a subset of stars from the other arrays.*)

In [27]:
alltypes = ['O', 'B', 'A', 'F', 'G', 'K', 'M']
for i, element in enumerate(alltypes):
    is_type = spectral_type_hp == element
    print (f'There are {len(spectral_type_hp[is_type])} stars of type \
{element} with parallaxes')
    color = color_hp[is_type]
    median_col = np.median(color)
    print(f'Their median color is {median_col}')
    magnitude = vmag_hp[is_type] - 5*np.log10(distance_hp[is_type]/10)
    median_mag = np.median(magnitude)
    print (f'Their median magnitude is {median_mag}\n')
# YOUR CODE HERE

There are 11 stars of type O with parallaxes
Their median color is -0.07
Their median magnitude is -1.0692437480817816

There are 315 stars of type B with parallaxes
Their median color is -0.09
Their median magnitude is -0.1489437914419698

There are 686 stars of type A with parallaxes
Their median color is 0.08
Their median magnitude is 1.2532530717588748

There are 623 stars of type F with parallaxes
Their median color is 0.43
Their median magnitude is 2.9010086203349745

There are 555 stars of type G with parallaxes
Their median color is 0.87
Their median magnitude is 2.1110964736695967

There are 679 stars of type K with parallaxes
Their median color is 1.16
Their median magnitude is 1.0056392679984931

There are 177 stars of type M with parallaxes
Their median color is 1.6
Their median magnitude is 0.14696342579112542



There are a number of interesting things to notice here!  Assuming you have written your code correctly, you will see:
- The number of stars in each category rises, then falls.  It rises because the more massive, bluer stars are relatively rarer.  It falls because the redder stars are dimmer, so we are less likely to detect them.  The decreasing number of M stars is a signal that our sample is *incomplete* at those colors.
- The color increases (becomes redder) mostly monotonically.  This is a good sign because color is directly related to spectral type.  The only exception is for O stars, but there are so few of those that this is probably an issue of *small-number statistics.*
- The intrinsic brightness decreases (magnitude increases) from O to F, which makes sense, but then turns around and starts increasing again.  This is weird because main-sequence K and M stars should be much fainter.  However, if you have taken a class on stars you will recall that after they finish their time on the main sequence, some stars evolve into red giants, which are extremely bright!  The increasing brightness in the redder bins indicates that our sample is highly *biased* towards those brighter red giant stars, even though they are extremely rare relative to main-sequence K and M dwarfs.

So the take-away message here is: always play with and examine your data for weird trends, outliers, and biases before blindly performing calculations - otherwise you won't know what might be hiding in there!

**(2 points)** Help me calibrate: how difficult did you find this homework on scale of 1 to 5?  (1 = piece of cake, 2 = pretty straightforward, 3 = took some thought but not a problem, 4 = challenging but I can do it, 5 = too frustrating)

In [ ]:
4